## **Build Custom Container with Custom prediction Routine and push it to Artifacts Registry**

In [7]:
from google.cloud.aiplatform.prediction import LocalModel 
from src.custom_sam_predictor import CustomSamPredictor 
import os
import logging

In [8]:
logging.basicConfig(level=logging.INFO)

In [10]:
USER_SRC_DIR = "src" # @param {type:"string"} source code directory name

""" Replace it with the region you will be working in on GCP """
REGION = "us-west1" # @param {type:"string"} region

""" Replace it with your Repository name """
REPOSITORY = "sam-container"  # @param {type:"string"} Artifacts registry repository name

""" Replace it with your PROJECT ID """
PROJECT_ID = "ml-ops-segment-anything" # @param {type:"string"} GCP project ID

IMAGE = "sam-cpr-container"  # @param {type:"string"} Custom Container name

LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts" # @param {type:"string"} Directory with model artifacts, Required for local deployment

In [11]:
""" Remove all unused containers """
!docker system prune -f

Total reclaimed space: 0B


#### **Build custom container**

In [20]:
""" Build custom docker container """
local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CustomSamPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
    base_image= "pytorch/pytorch:1.13.1-cuda11.6-cudnn8-runtime" # Base image from docker hub
)

In [13]:
local_model.get_serving_container_spec()

image_uri: "us-west1-docker.pkg.dev/ml-ops-segment-anything/sam-container/sam-cpr-container"
predict_route: "/predict"
health_route: "/health"

In [14]:
""" INPUT JSON file to test ENDPOINT """
INPUT_FILE_WITH_PROMPTS = "/home/jupyter/ml-ops-segment-anything/vertexAI_online_predictions/input_with_prompts.jsonl"
INPUT_FILE_WITHOUT_PROMPTS = "/home/jupyter/ml-ops-segment-anything/vertexAI_online_predictions/input_without_prompts.jsonl"

#### **Deploying Locally**

In [29]:
""" Deploying the model locally and simultaneously hitting the endpoint """

with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
    gpu_count=1
    
) as local_endpoint:
    print("predicting")
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE_WITH_PROMPTS,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

INFO:google.cloud.aiplatform.prediction.local_endpoint:Got the project id from the global config: ml-ops-segment-anything.


predicting


In [15]:
""" Printing model response """
print(predict_response.json())

In [18]:
""" Push the container to ARTIFACTS REGISTRY """
local_model.push_image()

### **Testing the ENDPOINT**

In [19]:
""" Making a curl call to make sure the endpoint is working 
Replace
    PROJECT_ID, ENDPOINT_ID and INPUT_DATA_FILE with the correct values after deployment 
"""
!curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-west1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-west1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"